<a href="https://colab.research.google.com/github/Roopesh-DMahanthi/Store-Transaction-Imputations-Nielsen-Hackathon-/blob/master/Hack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Store Transaction Imputations Hackathon Solution**

Installing **Pandasql** library (if not installed)

In [0]:
!pip install pandasql

Importing necessary dependencies

**Numpy, Pandas, Pandasql, Pyplot from Matplotlib**


In [0]:
import numpy as np
import pandas as pd
import pandasql as sql
from matplotlib import pyplot as plt

Importing **Hackathon_Working_Data.csv** File using pandas into **Dataframe**

In [0]:
df=pd.read_csv("Hackathon_Working_Data.csv")

Checking the Dataframe imported using **head** function

In [158]:
df.head

<bound method NDFrame.head of       MONTH STORECODE  ...              MBRD                    BRD
0        M1        N1  ...              AMUL                   AMUL
1        M1        N1  ...            MELODY       MELODY CHOCOLATY
2        M1        N1  ...     CADBURY SHOTS          CADBURY SHOTS
3        M1        N1  ...        WAGH BAKRI     WAGH BAKRI INSTANT
4        M1        N1  ...              TATA              TATA SALT
...     ...       ...  ...               ...                    ...
26980    M3       N10  ...  PARLE KRACK JACK       PARLE KRACK JACK
26981    M3       N10  ...       PARLE MARIE            PARLE MARIE
26982    M3       N10  ...           PARLE-G                PARLE-G
26983    M3       N10  ...             COLIN                  COLIN
26984    M3       N10  ...         BISK FARM  BISK FARM JUST GINGER

[26985 rows x 14 columns]>

Checking for **NULL** values in the Dataset

In [159]:
print((df.isnull()==True).sum())

MONTH        0
STORECODE    0
DAY          0
BILL_ID      0
BILL_AMT     0
QTY          0
VALUE        0
PRICE        0
GRP          0
SGRP         0
SSGRP        0
CMP          0
MBRD         0
BRD          0
dtype: int64


Since there are **No NULL** values

Getting a New Dataframe to Check **BILL_AMT** value of every Unique **BILL_ID** to be equal to **Total Cost of Items Purchased of that respective BILL_ID (Sumation of VALUE)** and **Total cost of Each and Every Individual Item Purchased (Summation of QTY*PRICE of Item)**

In [98]:
sub_data =sql.sqldf("SELECT MONTH,STORECODE,DAY,BILL_ID,BILL_AMT,SUM(VALUE) AS 'SUMATION',SUM(QTY*PRICE) AS 'PCHECK' FROM df WHERE BILL_ID IN (SELECT DISTINCT(BILL_ID) FROM df) GROUP BY MONTH,STORECODE,DAY,BILL_ID;", globals())
print(sub_data)

      MONTH STORECODE  DAY BILL_ID  BILL_AMT  SUMATION  PCHECK
0        M1        N1    4    T375     225.0     225.0   225.0
1        M1        N1    4    T379      95.0      95.0    95.0
2        M1        N1    4    T381      10.0      10.0    10.0
3        M1        N1    4    T382     108.0     108.0   108.0
4        M1        N1    4    T384      19.0      19.0    19.0
...     ...       ...  ...     ...       ...       ...     ...
13486    M3        N9   18   T1235     625.0     625.0   625.0
13487    M3        N9   19   T1242      63.0      63.0    63.0
13488    M3        N9   19   T1243     270.0     270.0   270.0
13489    M3        N9   19   T1275    6040.0    6040.0  6040.0
13490    M3        N9   20   T1253      30.0      30.0    30.0

[13491 rows x 7 columns]


Importing **Hackathon_Validation_Data.csv** Dataset

In [0]:
vdf=pd.read_csv('Hackathon_Validation_Data.csv')

Getting the **Shape** of the Dataframe

In [151]:
vdf.shape

(2430, 4)

Creating a **sol** numpy array to store **final result** with column names as required

In [0]:
sol=np.array(['ID','TOTALVALUE'])

Running an **SQL Query** to get appropriate **TOTALSUM** for the item and other details mentioned across each **ID** of the Validation Dataset and pushing the result to **sol** numpy array

In [0]:
for i in range(vdf.shape[0]):
  temp=vdf.loc[i]
  qry="select sum(VALUE) as 'TVALUE' from df where STORECODE='"+temp['STORECODE']+"' and MONTH='"+temp['MONTH']+"' and GRP='"+temp['GRP']+"' ;"
  fin=sql.sqldf(qry,globals())
  if(fin['TVALUE'][0]==None):
    sol=np.vstack([sol,[temp['ID'],0]])
    #print([temp['ID'],0])
  else:
    sol=np.vstack([sol,[temp['ID'],fin['TVALUE'][0]]])
    #print([temp['ID'],fin['TVALUE'][0]])
  #print(i,end=' ')

**Printing sol numpy array**

In [153]:
sol

array([['ID', 'TOTALVALUE'],
       ['1112535', '0'],
       ['1112539.0', '70.0'],
       ...,
       ['1122243', '0'],
       ['1122247.0', '971.0'],
       ['1122251.0', '1255.0']], dtype='<U32')

**Converting sol numpy array to Pandas Dataframe**

In [0]:
frame=pd.DataFrame(data=sol[1:,:],columns=sol[0,:])

Checking the **Converted Dataframe**

In [161]:
frame.head()

,ID,TOTALVALUE
0,1112535,0
1,1112539.0,70.0
2,1112543.0,164.0
3,1112547.0,545.0
4,1112551.0,70.8


***Exporting Dataframe into a CSV File***

In [0]:
frame.to_csv('ans.csv',index=False)